In [2]:
import torch
import torch.nn as nn
from read_code import *
from DataAdapter import DataAdapter
import torch.utils.data as Data
from torch.optim.lr_scheduler import CosineAnnealingLR,MultiStepLR
from model import CNN
import time
from batch import cal_cnn_batch
import torch.optim as optim
from EarlyStopping import EarlyStopping
from Regularization import Regularization
from Dataset import AssembleDataset

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_path = '../data/train_set'
batch_size = 512
epochs = 10
learning_rate = 0.00001
patience = 10
folds = 5
lead = 0

In [8]:
res = get_signal(data_path,lead)
X,Y = gen_extra_cnn_X_Y(res,50,af_rate = 2)
dataset = AssembleDataset(X,Y,folds,seed = 0)

文件读取完成


In [9]:
for fold in range(folds):

    valid_X,valid_Y = dataset.get_fold_data()
    train_X,train_Y = dataset.get_res_data()

    train_set = DataAdapter(train_X, train_Y)
    valid_set = DataAdapter(valid_X, valid_Y)

    train_loader = Data.DataLoader(train_set,batch_size = batch_size,shuffle = True,num_workers = 0)
    valid_loader = Data.DataLoader(valid_set,batch_size = batch_size,shuffle = False,num_workers = 0)

    model = CNN()
    model.to(device)
    cnn_path = r'./model/CNN_best_model' + str(lead) + '_' + str(fold) +'.pt'
    model.load_state_dict(torch.load(cnn_path,map_location='cuda:0'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    early_stopping = EarlyStopping(patience, verbose=False)
    # clr = CosineAnnealingLR(optimizer,T_max = 32)
    clr = MultiStepLR(optimizer,[20,50],gamma=0.1)

    reg_loss = Regularization(model, 0.001)
    best_loss = 100

    for epoch in range(1,epochs + 1):
        time_all=0
        start_time = time.time()
        # 训练模型
        train_res = cal_cnn_batch(train_loader, model, criterion, device, optimizer, reg_loss, True)
        # 验证集测试模型
        clr.step()
        valid_res = cal_cnn_batch(valid_loader, model, criterion, device, optimizer, reg_loss, False)
        time_all = time.time()-start_time
        # 打印训练及测试结果
        print('- Epoch: %d - Train_loss: %.5f - Train_mean_acc: %.5f - Train_F1: %.5f - Val_loss: %.5f - Val_mean_acc: %5f - Val_F1: %.5f - T_Time: %.3f' \
            %(epoch,train_res['loss'],train_res['acc'],train_res['F1'],valid_res['loss'],valid_res['acc'],valid_res['F1'],time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        # 保存最优模型
        if valid_res['loss'] < best_loss:
            best_loss = valid_res['loss']
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            torch.save(model.state_dict(), r'./extra_model/CNN_best_model' + str(lead) + '_' + str(fold) +'.pt')

        early_stopping(valid_res['loss'], model)
            # 若满足 early stopping 要求
        if early_stopping.early_stop:
            print("Early stopping")
            # 结束模型训练
            break

    print('CNN Training Finished')
    dataset.step()


---------------regularization weight---------------
conv1.convt.weight
conv1.conv1.weight
conv1.conv2.weight
conv1.conv3.weight
conv1.bn1.weight
conv1.bn2.weight
conv1.bn3.weight
conv1.se.fc1.weight
conv1.se.fc2.weight
conv1.cam.conv.weight
conv2.convt.weight
conv2.conv1.weight
conv2.conv2.weight
conv2.conv3.weight
conv2.bn1.weight
conv2.bn2.weight
conv2.bn3.weight
conv2.se.fc1.weight
conv2.se.fc2.weight
conv2.cam.conv.weight
conv3.convt.weight
conv3.conv1.weight
conv3.conv2.weight
conv3.conv3.weight
conv3.bn1.weight
conv3.bn2.weight
conv3.bn3.weight
conv3.se.fc1.weight
conv3.se.fc2.weight
conv3.cam.conv.weight
conv4.convt.weight
conv4.conv1.weight
conv4.conv2.weight
conv4.conv3.weight
conv4.bn1.weight
conv4.bn2.weight
conv4.bn3.weight
conv4.se.fc1.weight
conv4.se.fc2.weight
conv4.cam.conv.weight
conv5.convt.weight
conv5.conv1.weight
conv5.conv2.weight
conv5.conv3.weight
conv5.bn1.weight
conv5.bn2.weight
conv5.bn3.weight
conv5.se.fc1.weight
conv5.se.fc2.weight
conv5.cam.conv.weight
gru

In [6]:
test_X,test_Y = dataset.get_test_set()
test_set = DataAdapter(test_X, test_Y)
test_loader = Data.DataLoader(test_set,batch_size = batch_size,shuffle = False,num_workers = 0)

result = cal_cnn_batch(test_loader, model, criterion, device, optimizer, reg_loss, False)
print('confusion_matrix:',result['confusion_matrix'])
print('acc:',result['acc'])

confusion_matrix: [[6383.   38.]
 [  43. 7641.]]
acc: 0.9943498883928571


In [ ]:
cnn0 = CNN()
cnn_path_0 = r'.\model\CNN_best_model0_0.pt'
cnn0.load_state_dict(torch.load(cnn_path_0,map_location='cuda:0'))
cnn1 = CNN()
cnn_path_1 = r'.\model\CNN_best_model1_0.pt'
cnn1.load_state_dict(torch.load(cnn_path_1,map_location='cuda:0'))